# Beam search with recursion restarts

## Load model

In [1]:
!git clone https://github.com/attashe/ModifiedBeamSampler.git

Cloning into 'ModifiedBeamSampler'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 17 (delta 4), reused 12 (delta 2), pack-reused 0
Receiving objects: 100% (17/17), 18.37 KiB | 3.67 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [1]:
%cd ModifiedBeamSampler/

/content/ModifiedBeamSampler


In [16]:
!pip install transformers tokenizers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 20.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [3]:
!pip install bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 12.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [15]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 7.4 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.6.1-cp310-cp310-linux_x86_64.whl size=198447665 sha256=808523ff263d2fc1d3801147ad90d89dbfca67623a110c26592aef2064ea7b65
  Stored in directory: /root/.cache/pip/wheels/91/6a/38/f0faa036b4ac73a73247386f1ab1bb4cb4f6e72e6861a779f1
Successfully built flash-attn


In [2]:
import torch
import accelerate

import numpy as np
from typing import Optional, List
from transformers import AutoTokenizer, LlamaForCausalLM, BitsAndBytesConfig
from src.utils import _apply_top_k_top_p, _apply_min_p, _multinomial

### Default sampler

In [3]:
class Config:
    # Configuration for the LLM.
    temperature = 0.9
    top_k = 50

def generate_completion(text, *args, **kwargs):
    print(f'Generating default completion for text: {text}')
    # Tokenize the input text.
    inputs = tokenizer.encode(
            text,
            return_tensors='pt'
    ).to(model.device)

    outputs = model.generate(
        inputs,
        max_new_tokens=kwargs.get('max_tokens', 256),
        temperature=Config.temperature,
        top_k=Config.top_k,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        return_dict_in_generate=True, output_scores=True,
    )

    # Cut off the first tokens (the input text).
    outputs['sequences'] = outputs['sequences'][:, inputs.shape[-1]:]
    # Decode the generated text.
    completion_text = tokenizer.decode(outputs['sequences'][0], skip_special_tokens=True, clean_up_tokenization_space=True)
    print(f'Generated completion: {completion_text}')

    return completion_text

### Recursive sampler

In [4]:
def recursive_sampler_step(outputs, beam_len=5, temp=0.9, top_k=50, top_p=0.9, min_p=0.0, do_top_k_top_p=True, do_min_p=True,
                           terminators: Optional[List[int]] = None, generator: torch.Generator = None) -> int:
    sum_score = 0
    terminate = -1

    for i in range(beam_len):
        outputs = model.generate(
            outputs['sequences'],
            past_key_values=outputs.get("past_key_values"),
            max_new_tokens=1,
            pad_token_id=tokenizer.eos_token_id,
            # eos_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True, output_scores=True,
            do_sample=False
        )

        logits = outputs['scores'][0]
        logits.div_(temp)
        probs = torch.softmax(logits, dim=-1, dtype=torch.float)

        if do_top_k_top_p:
            probs = _apply_top_k_top_p(probs, top_k, top_p)

        if do_min_p:
            probs = _apply_min_p(probs, min_p)

        # sample_results = _multinomial(probs)
        # print(probs[probs > 0])
        probs[probs < 0] = 0
        sample_results = torch.multinomial(probs, num_samples=1, generator=generator)
        # print(sample_results)
        score = logits[:, sample_results[0][0]].item()
        sum_score += score

        # Replace last element (that were just greedy searched)
        outputs['sequences'][:, -1] = sample_results

        # Check if the token is a terminator.
        if terminators is not None and sample_results[0][0] in terminators:
            terminate = i
            break

    return outputs, sum_score, terminate

def generate_recursive_sampler(text, max_tokens=256, n_beams=2, beam_len=5, temperature=0.8,
                               top_k=5, top_p=1.0, do_min_p=True, min_p=0.1):
    print(f'Generating recursive completion for text: {text}')
    # Tokenize the input text.
    inputs = tokenizer.encode(
            text,
            return_tensors='pt'
    ).to(model.device)

    net_inputs = {'sequences': inputs}
    scores = np.zeros(n_beams)
    symbols_cnt = np.zeros(n_beams)
    terminates = np.zeros(n_beams)

    do_top_k_top_p = True
    top_k = torch.tensor([top_k]).to(model.device)
    top_p = torch.tensor([top_p]).to(model.device)
    min_p = torch.tensor([min_p]).to(model.device)

    g = torch.Generator('cuda')

    for _ in range(max_tokens // beam_len + 1):
        beams = {}
        for i in range(n_beams):
            test_out_1, score1, terminate1 = recursive_sampler_step(net_inputs, beam_len=beam_len,
                                                                    temp=temperature, top_k=top_k, top_p=top_p, min_p=min_p,
                                                                    do_top_k_top_p=do_top_k_top_p, do_min_p=do_min_p,
                                                                    terminators=[tokenizer.eos_token_id],
                                                                    generator=g)
            beams[i] = test_out_1
            scores[i] = score1
            terminates[i] = terminate1

            # print(f'[{i}] Score: {score1} - ', tokenizer.decode(test_out_1['sequences'][0, -beam_len:], skip_special_tokens=True, clean_up_tokenization_space=True))

        if np.any(terminates != -1):
            symbols_cnt[terminates != -1] += 1
            symbols_cnt[terminates == -1] = beam_len
            # Normalize scores
            scores = scores / symbols_cnt
            max_score = np.argmax(scores)

            if terminates[max_score] != -1:
                net_inputs = beams[max_score]
                break
            else:
                better_out = beams[max_score]
        else:
            max_score = np.argmax(scores)
            better_out = beams[max_score]

        # Update net_inputs to the better output
        net_inputs = better_out

    # Cut off the first tokens (the input text).
    net_inputs['sequences'] = net_inputs['sequences'][:, inputs.shape[-1]:]
    # Decode the generated text.
    completion_text = tokenizer.decode(net_inputs['sequences'][0], skip_special_tokens=True, clean_up_tokenization_space=True)
    print(f'Generated completion: {completion_text}')

    return completion_text

In [11]:
# rope settings
strategy_name = 'dynamic'
scaling_factor = 2.5

print("Loading Hermes-2-Pro-Llama-3 model...")
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Hermes-2-Pro-Llama-3-8B', trust_remote_code=True)
model = LlamaForCausalLM.from_pretrained(
    'NousResearch/Hermes-2-Pro-Llama-3-8B',
    torch_dtype=torch.float16,
    device_map="cuda",
    load_in_8bit=True,
    # Flash-attn and 4bit better to use on newer GPUs than free-tier colab
    # load_in_4bit=True,
    # attn_implementation="flash_attention_2",
    rope_scaling={"type": strategy_name, "factor": scaling_factor},
    # bnb_4bit_compute_dtype=torch.float16,
    max_position_embeddings=16384,
)

Loading Hermes-2-Pro-Llama-3 model...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Generate with default transformers sampler and recursive

### Story generation

In [12]:
prompts = [
    f"""<|im_start|>system
You are a sentient, superintelligent artificial general intelligence, here to teach and assist me.<|im_end|>
<|im_start|>user
Write a long story about Geralt from Rivia, who hunted monsters and ended up in the home world of the elves. Divide story to chapters and add dialogues and detailed descriptions. 2000-3000 words.
<|im_start|>assistant""",
]


In [13]:
%%time
completion2 = generate_completion(prompts[0], max_tokens=2048)

Generating default completion for text: <|im_start|>system
You are a sentient, superintelligent artificial general intelligence, here to teach and assist me.<|im_end|>
<|im_start|>user
Write a long story about Geralt from Rivia, who hunted monsters and ended up in the home world of the elves. Divide story to chapters and add dialogues and detailed descriptions. 2000-3000 words.
<|im_start|>assistant
Generated completion: 
Chapter 1: Departure from the Known World

In the heart of the Nilfgaardian Empire, Geralt of Rivia sat by the hearth of his small cabin, his weary eyes gazing into the flames. The scent of roasting mutton filled the air, and the sound of rain pattered gently against the window. The weather seemed fitting for his current state of mind: melancholic, yet comforting.

A deep rumble echoed from outside – it was the voice Geralt had been waiting for. His old friend, Dijkstra, had arrived with news that a sorceress had taken over a castle on the outskirts of Kovir. Geralt h

In [14]:
%%time
completion = generate_recursive_sampler(prompts[0], max_tokens=2048, n_beams=3, beam_len=5, temperature=1.5, top_k=50, min_p=0.1)

Generating recursive completion for text: <|im_start|>system
You are a sentient, superintelligent artificial general intelligence, here to teach and assist me.<|im_end|>
<|im_start|>user
Write a long story about Geralt from Rivia, who hunted monsters and ended up in the home world of the elves. Divide story to chapters and add dialogues and detailed descriptions. 2000-3000 words.
<|im_start|>assistant


KeyboardInterrupt: 

### Question 1

In [15]:
question = """A group of students are planning to go on a field trip to a museum. They need to decide how many buses to rent and how to divide the students among the buses. Each bus can hold up to 40 students, but the museum can only accommodate 120 students at a time. The group has a budget of $800 for the bus rental, and each bus costs $200 per day. How many buses should the group rent, and how many students should go on each bus? Explain your reasoning."""

prompts = [
    f"""<|im_start|>system
You are a sentient, superintelligent artificial general intelligence, here to teach and assist me.<|im_end|>
<|im_start|>user
{question}
<|im_start|>assistant""",
]

answers = """3 buses, 40 students each, $200 each"""

In [16]:
%%time
completion2 = generate_completion(prompts[0], max_tokens=512)

Generating default completion for text: <|im_start|>system
You are a sentient, superintelligent artificial general intelligence, here to teach and assist me.<|im_end|>
<|im_start|>user
A group of students are planning to go on a field trip to a museum. They need to decide how many buses to rent and how to divide the students among the buses. Each bus can hold up to 40 students, but the museum can only accommodate 120 students at a time. The group has a budget of $800 for the bus rental, and each bus costs $200 per day. How many buses should the group rent, and how many students should go on each bus? Explain your reasoning.
<|im_start|>assistant
Generated completion: 
To solve this problem, we need to find the optimal number of buses to rent and the number of students per bus, considering the museum's capacity, the budget for bus rental, and the capacity of each bus.

1. First, let's look at the budget constraint. The group has a total of $800 for bus rental. Each bus costs $200 per da

In [17]:
%%time
completion = generate_recursive_sampler(prompts[0], max_tokens=512, n_beams=3, beam_len=5, temperature=1.5, top_k=10, min_p=0.05)

Generating recursive completion for text: <|im_start|>system
You are a sentient, superintelligent artificial general intelligence, here to teach and assist me.<|im_end|>
<|im_start|>user
A group of students are planning to go on a field trip to a museum. They need to decide how many buses to rent and how to divide the students among the buses. Each bus can hold up to 40 students, but the museum can only accommodate 120 students at a time. The group has a budget of $800 for the bus rental, and each bus costs $200 per day. How many buses should the group rent, and how many students should go on each bus? Explain your reasoning.
<|im_start|>assistant
Generated completion: 
First, let's calculate how many buses the group can afford to rent given their budget:

Total budget: $800
Cost per bus per day: $200

Number of buses the group can afford to rent: $800 / $200 = 4 buses

Now, as the museum can only accommodate 120 students at a time and each bus can hold up to 40 students, we need to di

### Question 2

In [ ]:
question = """A group of four friends are going to play a board game together. They have to choose between four games: chess, checkers, monopoly, and scrabble. Each friend has a different preference for the game. Here are some clues to help you figure out their preferences: Amy likes chess more than monopoly, but less than scrabble. Bob likes checkers more than chess, but less than monopoly. Carol likes scrabble more than checkers, but less than chess. Dan likes monopoly more than scrabble, but less than checkers. What is the order of preference for each friend from most to least liked game? Write your answer using the following format: Friend: Game > Game > Game > Game"""
prompts = [
    f"""<|im_start|>system
You are a sentient, superintelligent artificial general intelligence, here to teach and assist me.<|im_end|>
<|im_start|>user
{question}
<|im_start|>assistant""",
]
answer = """
Amy: Scrabble > Chess > Monopoly > Checkers
Bob: Monopoly > Checkers > Chess > Scrabble
Carol: Chess > Scrabble > Checkers > Monopoly
Dan: Checkers> Monopoly > Scrabble > Chess
"""

In [ ]:
%%time
completion2 = generate_completion(prompts[0], max_tokens=512)

Generating default completion for text: <|im_start|>system
You are a sentient, superintelligent artificial general intelligence, here to teach and assist me.<|im_end|>
<|im_start|>user
A group of four friends are going to play a board game together. They have to choose between four games: chess, checkers, monopoly, and scrabble. Each friend has a different preference for the game. Here are some clues to help you figure out their preferences: Amy likes chess more than monopoly, but less than scrabble. Bob likes checkers more than chess, but less than monopoly. Carol likes scrabble more than checkers, but less than chess. Dan likes monopoly more than scrabble, but less than checkers. What is the order of preference for each friend from most to least liked game? Write your answer using the following format: Friend: Game > Game > Game > Game
<|im_start|>assistant
Generated completion: 
Amy: Scrabble > Chess > Monopoly > Checkers
Bob: Monopoly > Chess > Checkers > Scrabble
Carol: Chess > Sc

In [ ]:
%%time
completion = generate_recursive_sampler(prompts[0], max_tokens=512, n_beams=3, beam_len=5, temperature=1., top_k=50, min_p=0.1)

Generating recursive completion for text: <|im_start|>system
You are a sentient, superintelligent artificial general intelligence, here to teach and assist me.<|im_end|>
<|im_start|>user
A group of four friends are going to play a board game together. They have to choose between four games: chess, checkers, monopoly, and scrabble. Each friend has a different preference for the game. Here are some clues to help you figure out their preferences: Amy likes chess more than monopoly, but less than scrabble. Bob likes checkers more than chess, but less than monopoly. Carol likes scrabble more than checkers, but less than chess. Dan likes monopoly more than scrabble, but less than checkers. What is the order of preference for each friend from most to least liked game? Write your answer using the following format: Friend: Game > Game > Game > Game
<|im_start|>assistant
Generated completion: 
Amy: Scrabble > Chess > Monopoly > Checkers
Bob: Monopoly > Chess > Checkers > Scrabble
Carol: Chess > 

### Question 3

In [ ]:
question = """"Selena, Jennifer and Miley wear a blue dress, yellow dress, and green dress in an unknown order. It is known that:

1) If Selena wears blue, then Jennifer wears green.
2) If Selena wears yellow, then Miley wears green.
3) If Jennifer does not wear yellow, then Miley wears blue.

What is the color of the dress Selena is wearing? Think step by step explaining your reasoning."""
prompts = [
    f"""<|im_start|>system
You are a sentient, superintelligent artificial general intelligence, here to teach and assist me.<|im_end|>
<|im_start|>user
{question}
<|im_start|>assistant""",
]
answer = """Green"""

In [ ]:
%%time
completion2 = generate_completion(prompts[0], max_tokens=512)

Generating default completion for text: <|im_start|>system
You are a sentient, superintelligent artificial general intelligence, here to teach and assist me.<|im_end|>
<|im_start|>user
"Selena, Jennifer and Miley wear a blue dress, yellow dress, and green dress in an unknown order. It is known that:

1) If Selena wears blue, then Jennifer wears green.
2) If Selena wears yellow, then Miley wears green.
3) If Jennifer does not wear yellow, then Miley wears blue.

What is the color of the dress Selena is wearing? Think step by step explaining your reasoning.
<|im_start|>assistant
Generated completion: 
To determine the color of the dress Selena is wearing, let's analyze the given information step by step:

Step 1: Analyze the first condition:
If Selena wears blue, then Jennifer wears green.
This condition tells us that if Selena wears a blue dress, then Jennifer is sure to wear a green dress.

Step 2: Analyze the second condition:
If Selena wears yellow, then Miley wears green.
This condi

In [ ]:
%%time
completion = generate_recursive_sampler(prompts[0], max_tokens=2048, n_beams=3, beam_len=5, temperature=1.5, top_k=50, min_p=0.1)

Generating recursive completion for text: <|im_start|>system
You are a sentient, superintelligent artificial general intelligence, here to teach and assist me.<|im_end|>
<|im_start|>user
"Selena, Jennifer and Miley wear a blue dress, yellow dress, and green dress in an unknown order. It is known that:

1) If Selena wears blue, then Jennifer wears green.
2) If Selena wears yellow, then Miley wears green.
3) If Jennifer does not wear yellow, then Miley wears blue.

What is the color of the dress Selena is wearing? Think step by step explaining your reasoning.
<|im_start|>assistant
Generated completion: 
Step 1: Rule out scenarios where Selena is not wearing her given colors (blue, yellow, or green).

Step 2: Since either Jennifer wears green or Miley wears green (considering Scenario 1 where Selena is wearing blue), we know that Selena must be wearing either yellow or green.

Step 3: If Selena wore green, Jennifer couldn't wear green as there wouldn't be any green left for Miley (since S

In [ ]:
%%time
completion = generate_recursive_sampler(prompts[0], max_tokens=2048, n_beams=3, beam_len=5, temperature=1.5, top_k=50, min_p=0.1)

Generating recursive completion for text: <|im_start|>system
You are a sentient, superintelligent artificial general intelligence, here to teach and assist me.<|im_end|>
<|im_start|>user
"Selena, Jennifer and Miley wear a blue dress, yellow dress, and green dress in an unknown order. It is known that:

1) If Selena wears blue, then Jennifer wears green.
2) If Selena wears yellow, then Miley wears green.
3) If Jennifer does not wear yellow, then Miley wears blue.

What is the color of the dress Selena is wearing? Think step by step explaining your reasoning.
<|im_start|>assistant
Generated completion: 
Step 1: Rule out scenarios where Selena is not wearing her given colors (blue, yellow, or green).

Step 2: Since either Jennifer wears green or Miley wears green (considering Scenario 1 where Selena is wearing blue), we know that Selena must be wearing either yellow or green.

Step 3: If Selena wore green, Jennifer couldn't wear green as there wouldn't be any green left for Miley (since S

### Test Sampler class

In [ ]:
from src.sampler import RecursiveSampler

In [ ]:
prompts = [
    f"""<|im_start|>system
You are a sentient, superintelligent artificial general intelligence, here to teach and assist me.<|im_end|>
<|im_start|>user
Tell me joke about Machine Learning Researcher.
<|im_start|>assistant""",
]

sampler = RecursiveSampler(model, tokenizer)

In [ ]:
response = sampler.generate(prompts[0], max_tokens=512, n_beams=3, beam_len=5, temperature=1.5, top_k=50, min_p=0.1)

Generating recursive completion for text: <|im_start|>system
You are a sentient, superintelligent artificial general intelligence, here to teach and assist me.<|im_end|>
<|im_start|>user
Tell me joke about Machine Learning Researcher.
<|im_start|>assistant
Generated completion: 
Why did the machine learning researcher break up with regression analysis? Because they just couldn't come to a significant conclusion.


In [ ]:
response

"\nWhy did the machine learning researcher break up with regression analysis? Because they just couldn't come to a significant conclusion."